# Life Cycle Assessment of the CeRAS aircraft

The aim of this tutorial is to evaluate the environmental impacts of the CeRAS aircraft using the LCA module of FAST-OAD. It is assumed that the data related to the CeRAS aircraft have already been retrieved with the CS-25 version of FAST-OAD. Therefore, the LCA module is applied here as a post-processing step.

More information on the CeRAS case study can be found [here](https://github.com/fast-aircraft-design/FAST-OAD_CS25/blob/main/src/fastoad_cs25/notebooks/02_CeRAS_case_study/CeRAS_case_study.ipynb).

## Imports

In [1]:
import os.path as pth
import sys
import logging
import fastoad.api as oad

sys.path.append(pth.abspath("."))

## Working directories and reference files

In [2]:
#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
#SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "CeRAS_reference_data.xml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "operational_missions_out.xml")

## 1. FAST-OAD configuration file

In [3]:
CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "oad_configuration.yml")

## 2. LCA file

In [4]:
LCA_FILE = pth.join(DATA_FOLDER_PATH, "lca_model.yml")

## 3. Inputs file

In [5]:
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True`.
Creating custom LCIA method  ('Custom methods', 'climate change (with non-CO2)', 'global warming potential (GWP100)')


Writing activities to SQLite3 database:
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/30/2025 19:04:17
  Finished: 01/30/2025 19:04:17
  Total time elapsed: 00:00:00
  CPU %: 2.60
  Memory %: 1.83
Added 1 new biosphere flows
1 methods
130 cfs
0 unlinked cfs
Wrote 1 LCIA methods with 130 characterization factors
Wrote matching file to:
/Users/f.pollet/POSTDOC/5 - Tools/FAST-OAD_LCA/src/fastoad_lca/notebooks/01_CeRAS/data/lcia_nonCO2_global_warming_updated.xlsx
Creating custom LCIA method  ('Custom methods', 'total: ecosystem quality (with non-CO2)', 'ecosystem quality')
1 methods
1990 cfs
0 unlinked cfs
Wrote 1 LCIA methods with 1990 characterization factors
Wrote matching file to:
/Users/f.pollet/POSTDOC/5 - Tools/FAST-OAD_LCA/src/fastoad_lca/notebooks/01_CeRAS/data/lcia_nonCO2_ecosystem_quality_updated.xlsx
Creating custom LCIA method  ('Custom methods', 'total: human health (with non-CO2)', 'human health')
1 methods
1618 cfs
0 unlinked cfs
Wrote 1 LCIA methods with 1618 characterization factors
Wrote matchin

In [6]:
oad.variable_viewer(input_file)

/opt/miniconda3/envs/fastoad-lca/lib/python3.10/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## 4. Run the LCA

In [7]:
LCA_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

## 5. Results Analysis

In [8]:
oad.variable_viewer(LCA_problem.output_file_path)